<a href="https://colab.research.google.com/github/TiFcode/tif-llm-2/blob/main/TiFcode_Chat_With_Your_Documents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TiFcode - Chat With Your Documents

First we get the secret Google GenAI API key from the Google Colab Secrets:

In [24]:
from google.colab import userdata as GoogleColabSecretUserData
SECRET_GOOGLE_GENAI_API_KEY=GoogleColabSecretUserData.get("GoogleGenAiApiKey")

In [25]:
!curl ipinfo.io

{
  "ip": "35.196.60.206",
  "hostname": "206.60.196.35.bc.googleusercontent.com",
  "city": "North Charleston",
  "region": "South Carolina",
  "country": "US",
  "loc": "32.8546,-79.9748",
  "org": "AS396982 Google LLC",
  "postal": "29415",
  "timezone": "America/New_York",
  "readme": "https://ipinfo.io/missingauth"
}

In [26]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [27]:
!pip install -q --upgrade google-generativeai langchain-google-genai langchain pypdf chromadb

In [28]:
import google.generativeai

In [29]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain

In [30]:
from IPython.display import Markdown
import textwrap

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [31]:
google.generativeai.configure(api_key=SECRET_GOOGLE_GENAI_API_KEY)

Now we list the names of the generative models that are available in the API, as we need to select one of them and configure a string with the name of the selected model:

In [32]:
for ai_model in google.generativeai.list_models():
  if 'generateContent' in ai_model.supported_generation_methods:
    print(ai_model.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-pro
models/gemini-pro-vision


In [105]:
llm = ChatGoogleGenerativeAI(model="gemini-1.0-pro", temperature=0.3, google_api_key=SECRET_GOOGLE_GENAI_API_KEY)

#Prepare the PDF documents that we will chat on with the AI

In [34]:
!mkdir pdfs
!gdown 1v94CNKi1truPO2M1E0l1VTDGxiqdRcRk -O "pdfs/OntoSense - Whitepaper (version 2018_01_06__11_09).pdf"


mkdir: cannot create directory ‘pdfs’: File exists
Downloading...
From: https://drive.google.com/uc?id=1v94CNKi1truPO2M1E0l1VTDGxiqdRcRk
To: /content/pdfs/OntoSense - Whitepaper (version 2018_01_06__11_09).pdf
100% 1.66M/1.66M [00:00<00:00, 89.8MB/s]


In [35]:
document_loader = PyPDFDirectoryLoader("pdfs")
loaded_pages = document_loader.load_and_split()

In [36]:
print("Number of loaded pages:", len(loaded_pages))

# Length of each loaded page
# *** TBD

Number of loaded pages: 106


In [37]:
#print("The first loaded page is:\n\n", loaded_pages[0].page_content)

#Prepare the text chunks

In [38]:
all_text_from_loaded_pages = "\n\n".join(str(loaded_page.page_content) for loaded_page in loaded_pages)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=200)

text_chunks = text_splitter.split_text(all_text_from_loaded_pages)

In [39]:
print("Number of text chunks:", len(text_chunks))

Number of text chunks: 15


In [40]:
#print("The first text chunk is:\n\n[",text_chunks[0],"]")

#Compute the embeddings of the text chunks

In [41]:
google_genai_embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001", google_api_key=SECRET_GOOGLE_GENAI_API_KEY)
chroma_vector_store = Chroma.from_texts(text_chunks, google_genai_embeddings).as_retriever()

#Chatting

In [99]:
question = "what is CR and why is it useful"
relevant_text_chunks = chroma_vector_store.get_relevant_documents(question)

In [100]:
print("Number of relevant text chunks:", len(relevant_text_chunks))

Number of relevant text chunks: 4


In [101]:
#relevant_text_chunks[0]

In [102]:
prompt_template = """
  Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in provided context just say "answer is not available in the context", don't provide the wrong answer\n\n
  Context:\n {context}\n
  Question:\n {param_question}?\n

  Answer:
"""

prompt = PromptTemplate(template = prompt_template, input_variables = ["context","param_question"])

In [103]:
chain = load_qa_chain(llm, chain_type="stuff", prompt=prompt)
response = chain(
    {"input_documents":relevant_text_chunks,
     "param_question":question},
     return_only_outputs=True
)

In [104]:
print("The question was:\n\n[",question,"]\n\n")
text_to_format = response['output_text']
formatted_text = textwrap.fill(text_to_format, width=80)
print("The response is:\n\n[",formatted_text,"]")

The question was:

[ what is CR and why is it useful ]


The response is:

[ CR stands for Confidence Rank. It is a measure of the confidence of the system
in using a specific concept/node/edge (denoted c new).   It is useful because it
allows the system to evaluate the importance of a concept/node/edge based on its
source, references, goals, and feedback. This information can then be used to
make decisions about which concepts/nodes/edges to use in the system. ]
